In [1]:
##栅格数据与矢量数据的转换
from osgeo import gdal, ogr
import matplotlib.pyplot as plt

In [2]:
path_mndwi = 'E:\\开源\\HW2\\neijiang_6bands_10m_extractor.tif'
dset = gdal.Open(path_mndwi)
##提取栅格图像的数组
mndwi = dset.GetRasterBand(1)##mndwi该数组只有一个波段
mndwi_array = mndwi.ReadAsArray()
print(mndwi_array.shape)

(1806, 1979)


In [3]:
path_mndwipolygon = 'E:\\开源\\HW2\\neijiang_shp.gpkg'

proj = gdal.osr.SpatialReference()
proj.ImportFromWkt(dset.GetProjection())

driver_vec = ogr.GetDriverByName('GPKG')
ds_vec = driver_vec.CreateDataSource(path_mndwipolygon)
poly_layer = ds_vec.CreateLayer('mndwi_polygon', srs=proj, geom_type=ogr.wkbMultiPolygon)
##栅格与矢量的空间参考一样，就从栅格读入给矢量
Field_DN = ogr.FieldDefn('DN', ogr.OFTInteger)
poly_layer.CreateField(Field_DN)
##注意以下放大的三个参数：
gdal.Polygonize(srcBand=mndwi,maskBand=None,outLayer=poly_layer,iPixValField=0)

##删除非水体的面shp
DN_vec = [1]
for fea in poly_layer:
    DN_value = fea.GetField('DN')
    if DN_value not in DN_vec:
        poly_layer.DeleteFeature(fea.GetFID())
ds_vec = None
dset = None

In [4]:
path_vec = 'E:\\开源\\HW2\\neijiang_shp.gpkg'
path_Raster = 'E:\\开源\\HW2\\neijiang_Raster.gpkg'

In [5]:
ds_vec = ogr.Open(path_vec)
layer_vec =ds_vec.GetLayer()
proj_vec = layer_vec.GetSpatialRef()
##转为Wkt空间参考：
proj_vec_wkt = proj_vec.ExportToWkt()
print(proj_vec)

PROJCS["WGS 84 / UTM zone 48N",
    GEOGCS["WGS 84",
        DATUM["World Geodetic System 1984",
            SPHEROID["WGS 84",6378137,298.257223563]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",105],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH]]


In [6]:
x_min, x_max,y_min,y_max = layer_vec.GetExtent()
print(x_min,x_max,y_min,y_max)

489960.0 509750.0 3263940.0 3282000.0


In [9]:
pixel_size = 20
geo_trans = [x_min,20,0,y_max,0,-20]


x_size = int((x_max-x_min)/pixel_size)
y_size = int((y_max-y_min)/pixel_size)

driver = gdal.GetDriverByName('GTiff')
dset_raster = driver.Create(path_Raster,
                            xsize=x_size,
                            ysize=y_size,
                            bands=1,
                            eType=gdal.GDT_Int16)
dset_raster.SetGeoTransform(geo_trans)
dset_raster.SetProjection(proj_vec_wkt)


0

In [10]:
gdal.RasterizeLayer(dataset=dset_raster,bands=[1],layer=layer_vec, options=['ATTRIBUTE=DN'])
dset_raster = None